In [2]:
# import numpy as np
# import imutils
# import cv2
import time

In [ ]:
# Define the paths to the prototxt file and pre-trained model
prototxt_path = r"D:\PythonProject\Face_Recognition_DL\model\deploy.prototxt.txt"
model_path = r"D:\PythonProject\Face_Recognition_DL\model\res10_300x300_ssd_iter_140000.caffemodel"
confidence_threshold = 0.5

# Resize OpenCV window size
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)

# Load model architecture and weights
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# Initialize camera stream
print("[INFO] starting video stream...")

vid = cv2.VideoCapture(0)

# Loop over the frames from the video stream
while True:
    # Start the timer to calculate inference time
    start_time = time.time()

    # Read frame from camera and resize to 400 pixels
    ret, frame = vid.read()
    frame = imutils.resize(frame, width=400)
 
    # Grab the frame dimensions and convert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
 
    # Pass the blob through the network and obtain the detections and predictions
    net.setInput(blob)
    detections = net.forward()

    # Calculate inference time
    inference_time = time.time() - start_time

    # Add inference time text on top of the frame
    cv2.putText(frame, "Inference Time: {:.4f} sec".format(inference_time), (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)

    # Loop over the detections
    for i in range(0, detections.shape[2]):
        # Extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]

        # Filter out detections by confidence
        if confidence < confidence_threshold:
            continue

        # Compute the (x, y)-coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
 
        # Draw the bounding box of the face along with the associated probability
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY),
                      (0, 0, 255), 1)
        cv2.putText(frame, text, (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)

    # Show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# After the loop, release the cap object and destroy all windows
vid.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import time

# Define the paths to the prototxt file and pre-trained model
prototxt_path = r"D:\PythonProject\Face_Recognition_DL\model\deploy.prototxt.txt"
model_path = r"D:\PythonProject\Face_Recognition_DL\model\res10_300x300_ssd_iter_140000.caffemodel"
confidence_threshold = 0.5

# Load model architecture and weights
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# Load the image
image_path = r"D:\PythonProject\Face_Recognition_DL\Multiple_selfie.jpg"  # Change this to your image path
image = cv2.imread(image_path)
image = cv2.resize(image, (600, 500))

# Grab the image dimensions
(h, w) = image.shape[:2]

# Convert image to a blob
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
                             (300, 300), (104.0, 177.0, 123.0))

# Pass the blob through the network and obtain the detections
net.setInput(blob)
detections = net.forward()

# Loop over the detections
for i in range(0, detections.shape[2]):
    # Extract the confidence associated with the prediction
    confidence = detections[0, 0, i, 2]

    # Filter out detections by confidence
    if confidence < confidence_threshold:
        continue

    # Compute the (x, y)-coordinates of the bounding box
    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")

    # Draw the bounding box and probability
    text = "{:.2f}%".format(confidence * 100)
    y = startY - 10 if startY - 10 > 10 else startY + 10
    cv2.rectangle(image, (startX, startY), (endX, endY),
                  (0, 0, 255), 1)
    cv2.putText(image, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

# Show the output image
cv2.imshow("Face Detection", image)
cv2.imwrite(r"D:\PythonProject\Face_Recognition_DL\Face_detection_test_op\SSD.jpg", image)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import cv2
import numpy as np
import os

# Define paths to model files
prototxt_path = r"D:\PythonProject\Face_Recognition_DL\model\deploy.prototxt.txt"
model_path = r"D:\PythonProject\Face_Recognition_DL\model\res10_300x300_ssd_iter_140000.caffemodel"

# Confidence threshold for face detection
confidence_threshold = 0.5

# Create a directory to save images if it doesn't exist
output_dir = r"Rifat Banu"
os.makedirs(output_dir, exist_ok=True)

# Load the Caffe face detection model
print("[INFO] Loading model...")
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# Initialize webcam
print("[INFO] Starting video stream...")
vid = cv2.VideoCapture(0)

# Image capture settings
img_count = 0
batch_size = 100
total_images = 500

while img_count < total_images:
    captured_this_batch = 0
    print(f"Starting capture for batch {img_count // batch_size + 1}")

    while captured_this_batch < batch_size and img_count < total_images:
        ret, frame = vid.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Resize frame for better processing
        frame = cv2.resize(frame, (600, 500))
        (h, w) = frame.shape[:2]

        # Preprocess frame for the face detection model
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence < confidence_threshold:
                continue

            # Get bounding box coordinates
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Crop and validate face detection
            face_cropped = frame[startY:endY, startX:endX]
            if face_cropped.shape[0] < 10 or face_cropped.shape[1] < 10:
                continue  # Ignore very small detections

            # Resize to 96x96 for OpenFace compatibility
            face_resized = cv2.resize(face_cropped, (96, 96))

            # Convert to grayscale
            face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)

            # Save the grayscale face image
            img_filename = os.path.join(output_dir, f"face_{img_count+1}.jpg")
            cv2.imwrite(img_filename, face_gray)
            img_count += 1
            captured_this_batch += 1

            # Draw bounding box on frame
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 1)
            cv2.putText(frame, f"Captured: {img_count}/{total_images}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            # Stop capturing when batch is complete
            if captured_this_batch >= batch_size or img_count >= total_images:
                break

        # Show the live video feed
        cv2.imshow("Face Capture", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    print("Press 'c' to capture the next batch of images, or 'q' to stop.")
    key = cv2.waitKey(0) & 0xFF
    if key == ord('q'):
        break

# Release resources
vid.release()
cv2.destroyAllWindows()
print(f"[INFO] Captured {img_count} images, saved in '{output_dir}' directory.")


[INFO] Loading model...
[INFO] Starting video stream...
Starting capture for batch 1
Press 'c' to capture the next batch of images, or 'q' to stop.
Starting capture for batch 2
Press 'c' to capture the next batch of images, or 'q' to stop.
Starting capture for batch 3
Press 'c' to capture the next batch of images, or 'q' to stop.
Starting capture for batch 4
Press 'c' to capture the next batch of images, or 'q' to stop.
Starting capture for batch 5
Press 'c' to capture the next batch of images, or 'q' to stop.
[INFO] Captured 500 images, saved in 'Rifat Banu' directory.


In [2]:
pip install basicsr facexlib gfpgan

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
